<a href="https://colab.research.google.com/github/kimpitang/AI-Agent/blob/main/Langchain%EC%9C%BC%EB%A1%9C_RAG_%EA%B5%AC%EC%B6%95%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#PyPDF 설치
!pip install -q pypdf langchain langchain langchain_community faiss-cpu langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


In [4]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['OPENAI_API_KEY'] = "" // 개인 API 키

### Document Loader로 문서 불러오기

In [5]:
# PyPDF Loader 불러오기
from langchain_community.document_loaders import PyPDFLoader

# PDF파일 불러올 객체 PyPDFLoader 선언
loader = PyPDFLoader(r"/content/Retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks-Paper.pdf")

# PDF파일 로드 및 페이지별로 자르기
pages = loader.load_and_split()

In [6]:
pages[0]

Document(metadata={'source': '/content/Retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks-Paper.pdf', 'page': 0, 'page_label': '1'}, page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡ , Ethan Perez?,\nAleksandra Piktus† , Fabio Petroni† , Vladimir Karpukhin† , Naman Goyal† , Heinrich Küttler† ,\nMike Lewis† , Wen-tau Yih† , Tim Rocktäschel†‡ , Sebastian Riedel†‡ , Douwe Kiela†\n† Facebook AI Research;‡ University College London;?New York University;\nplewis@fb.com\nAbstract\nLarge pre-trained language models have been shown to store factual knowledge\nin their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-\nstream NLP tasks. However, their ability to access and precisely manipulate\nknowledge is still limited, and hence on knowledge-intensive tasks, their perfor-\nmance lags behind task-speciﬁc architectures. Additionally, providing provenance\nfor their decisions and updating their world knowledge re

In [7]:
[(i.page_content[:100], i.metadata) for i in pages]

[('Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡ , Ethan Perez?,\nAle',
  {'source': '/content/Retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks-Paper.pdf',
   'page': 0,
   'page_label': '1'}),
 ('The\x03DiYineComed\\\x03(x) T\nQXeU\\EQcRdeU\nT([)\nMIPS pθ\nGeneUaWoU¬Sѡ(PaUaPeWULc)\nMaUgin-ali]eThiV\x0314Wh\x03cenW',
  {'source': '/content/Retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks-Paper.pdf',
   'page': 1,
   'page_label': '2'}),
 ('2 Methods\nWe explore RAG models, which use the input sequencex to retrieve text documentsz and use t',
  {'source': '/content/Retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks-Paper.pdf',
   'page': 1,
   'page_label': '2'}),
 ('by ✓ that generates a current token based on a context of the previousi \x00 1 tokens y1:i\x001, the origi',
  {'source': '/content/Retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks-Paper.pdf',
   'page': 2,
   'page_label': '3

In [9]:
from langchain_community.document_loaders import WebBaseLoader
# 텍스트 추출할 URL 입력
loader = WebBaseLoader("https://www.espn.com/")
# ssl verification 에러 방지를 위한 코드
loader.requests_kwargs = {'verify':False}
data = loader.load()
data

[Document(metadata={'source': 'https://www.espn.com/', 'title': 'ESPN - Serving Sports Fans. Anytime. Anywhere.', 'description': 'Visit ESPN for live scores, highlights and sports news. Stream exclusive games on ESPN+ and play fantasy sports.', 'language': 'en'}, page_content="\n\n\n\n\n\n\n\n\nESPN - Serving Sports Fans. Anytime. Anywhere.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        Skip to main content\n    \n\n        Skip to navigation\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<\n\n>\n\n\n\n\n\n\n\n\n\nMenuESPN\n\n\n\n\n\nscores\n\n\n\nNFLNBANCAAFNHLNCAAMNCAAWSoccerMore SportsBoxingCFLNCAACricketF1GamingGolfHorseLLWSMLBMMANASCARNLLNBA G LeagueNBA Summer LeagueNWSLOlympicsPLLProfessional WrestlingRacingRN BBRN FBRugbySports BettingTennisWNBAX GamesUFLEditionsFantasyWatchESPN BETESPN+\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

In [10]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
# 텍스트 추출할 URL 입력
loader = WebBaseLoader("https://www.espn.com/",
                       bs_kwargs=dict(
                           parse_only=bs4.SoupStrainer(
                               class_=("headlineStack top-headlines")
                           )
                       ))

# ssl verification 에러 방지를 위한 코드
loader.requests_kwargs = {'verify':False}
data = loader.load()
data

[Document(metadata={'source': 'https://www.espn.com/'}, page_content="Eagles' Moore: Focused on SB, not on Saints talkRams seeking to trade 'highly motivated' KuppSpurs tout Fox, Wemby pairing: 'Who do you stop?'Goodell: NFL 'better' because of diversity effortsMLB fires ump Hoberg for shared betting accountWild's Hartman gets 10 games for hit on St√ºtzleSources: Gambling ring also tied to college hoopsTrump hosts Panthers, lauds 'tremendous' Cup runTrade grades: Does Fox fit with Wemby? Top HeadlinesEagles' Moore: Focused on SB, not on Saints talkRams seeking to trade 'highly motivated' KuppSpurs tout Fox, Wemby pairing: 'Who do you stop?'Goodell: NFL 'better' because of diversity effortsMLB fires ump Hoberg for shared betting accountWild's Hartman gets 10 games for hit on St√ºtzleSources: Gambling ring also tied to college hoopsTrump hosts Panthers, lauds 'tremendous' Cup runTrade grades: Does Fox fit with Wemby?")]

### Text Splitter로 문서 분할하기

In [14]:
# PyPDFium2Loader로 PDF 문서 로드하기
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(r"/content/LLM_agent.pdf")
pages = loader.load()

# CharacterTextSplitter 모듈 로드
from langchain_text_splitters import character

# 구분자: 줄넘김, 청크 길이: 500, 청크 오버랩: 100, length_function: 글자수
text_splitter = character.CharacterTextSplitter(
    separator = "\n",
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

# 텍스트 분할
texts = text_splitter.split_documents(pages)
print(texts[0])

page_content='Published as a conference paper at COLM 2024
AutoGen: Enabling Next-Gen LLM Applications
via Multi-Agent Conversations
Qingyun Wu∗
Penn State University
qingyun@autogen.team
Gagan Bansal
Microsoft Research
gaganbansal@microsoft.com
Jieyu Zhang
University of Washington
jieyuz2@cs.washington.edu
Yiran Wu
Penn State University
yiran.wu@psu.edu
Beibin Li
Microsoft Research
beibin.li@microsoft.com
Erkang Zhu
Microsoft Research
erkang.zhu@microsoft.com
Li Jiang
Microsoft
lijiang1@microsoft.com' metadata={'source': '/content/LLM_agent.pdf', 'page': 0, 'page_label': '1'}


In [15]:
print([len(i.page_content) for i in texts])

[492, 473, 457, 438, 496, 462, 326, 499, 455, 498, 435, 466, 447, 429, 425, 427, 475, 434, 475, 455, 451, 442, 434, 478, 492, 421, 436, 443, 465, 444, 428, 440, 473, 464, 454, 438, 449, 206, 431, 444, 446, 470, 487, 429, 412, 456, 456, 428, 420, 432, 429, 418, 454, 452, 440, 446, 449, 493, 498, 477, 455, 412, 468, 463, 438, 437, 442, 369, 477, 459, 431, 443, 453, 441, 450, 470, 445, 445, 460, 435, 301, 435, 451, 438, 440, 492, 475, 428, 434, 425, 496, 438, 447, 184, 425, 451, 446, 433, 459, 451, 493, 426, 445, 444, 446, 443, 491, 481, 480, 458, 499, 461, 448, 368, 423, 466, 457, 491, 486, 457, 438, 478, 184, 420, 496, 417, 485, 443, 483, 414, 458, 207, 442, 484, 468, 434, 490, 450, 460, 474, 255, 436, 433, 500, 437, 492, 456, 388, 453, 423, 447, 429, 452, 481, 480, 238, 417, 489, 442, 443, 422, 466, 486, 450, 479, 489, 182, 428, 477, 486, 450, 446, 442, 423, 472, 413, 464, 500, 74, 426, 426, 434, 482, 421, 420, 444, 440, 425, 486, 491, 274, 496, 467, 434, 469, 458, 448, 450, 447, 485, 

In [17]:
# RecursiveCharacterTextSplitter는 구분자를 설정하지 않아도 chunk_size에 맞춰서 분리함

from langchain_community.document_loaders import PyPDFium2Loader

loader = PyPDFLoader(r"/content/LLM_agent.pdf")

pages = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter (
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

texts = text_splitter.split_documents(pages)
print([len(i.page_content) for i in texts])

[492, 473, 457, 438, 496, 462, 326, 499, 455, 498, 435, 466, 447, 429, 425, 427, 475, 434, 475, 455, 451, 442, 434, 478, 492, 421, 436, 443, 465, 444, 428, 440, 473, 464, 454, 438, 449, 206, 431, 444, 446, 470, 487, 429, 412, 456, 456, 428, 420, 432, 429, 418, 454, 452, 440, 446, 449, 493, 498, 477, 455, 412, 468, 463, 438, 437, 442, 369, 477, 459, 431, 443, 453, 441, 450, 470, 445, 445, 460, 435, 301, 435, 451, 438, 440, 492, 475, 428, 434, 425, 496, 438, 447, 184, 425, 451, 446, 433, 459, 451, 493, 426, 445, 444, 446, 443, 491, 481, 480, 458, 499, 461, 448, 368, 423, 466, 457, 491, 486, 457, 438, 478, 184, 420, 496, 417, 485, 443, 483, 414, 458, 207, 442, 484, 468, 434, 490, 450, 460, 474, 255, 436, 433, 448, 488, 472, 482, 444, 241, 453, 423, 447, 429, 452, 481, 480, 238, 417, 489, 442, 443, 422, 466, 486, 450, 479, 489, 182, 428, 477, 486, 450, 446, 442, 423, 472, 413, 464, 428, 161, 426, 426, 434, 482, 421, 420, 444, 440, 425, 486, 491, 274, 496, 467, 434, 469, 458, 448, 450, 447,

### 임베딩 모델로 텍스트를 수치화하기

In [18]:
import os
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model = 'text-embedding-3-small')
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

(5, 1536)

In [19]:
print(embeddings[0][:10])
print("-"*50)
print(f"임베딩 개수: len(embeddings) \n 이베딩 차원: len(embeddings[0])")

[-0.019187437370419502, -0.03813096508383751, -0.031003428623080254, -0.004613928031176329, -0.03536667302250862, -0.004004158079624176, 0.012920353561639786, 0.05103099346160889, -0.005826693493872881, -0.03720953315496445]
--------------------------------------------------
임베딩 개수: len(embeddings) 
 이베딩 차원: len(embeddings[0])


In [20]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 임베딩 모델 API 호출
embeddings_model = OpenAIEmbeddings(model = 'text-embedding-3-small')

# PDF 문서 로드
loader = PyPDFLoader(r"/content/LLM_agent.pdf")
pages = loader.load()

# PDF 문서르 여러 청크로 분할
text_splitter = RecursiveCharacterTextSplitter (
    chunk_size = 500,
    chunk_overlap  = 100
)

texts = text_splitter.split_documents(pages)

# OpenAI 임베딩 모델로 청크들을 임베딩 변환하기
embeddings = embeddings_model.embed_documents([i.page_content for i in texts])
len(embeddings), len(embeddings[0])

(445, 1536)

### 벡터 스토어에 저장

In [21]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(texts, embeddings_model)

### Retriever 생성

In [22]:
retriever = db.as_retriever()

query = "autogen이 뭐야?"

# 유사 문서 검색
retriever.invoke(query)

[Document(id='b9d518fe-6ada-4037-912f-69daab5ab1a3', metadata={'source': '/content/LLM_agent.pdf', 'page': 14, 'page_label': '15'}, page_content='• A repository containing the source code of AutoGen (v0.1.1) and application evaluation\ncode is provided, along with instructions for reproducing the experiments reported in\nthis paper: https://github.com/qingyun-wu/autogen/tree/2024-03.\n15'),
 Document(id='e5eea139-1af7-4097-ab0a-a6b9fd0a113a', metadata={'source': '/content/LLM_agent.pdf', 'page': 19, 'page_label': '20'}, page_content='autonomous mode AutoGen supports could be desirable in many scenarios, a high level of\nautonomy can also pose potential risks, especially in high-risk applications (Amodei et al.,\n2016; Weld & Etzioni, 1994). As a result, building fail-safes against cascading failures and ex-\nploitation, mitigating reward hacking, out of control and undesired behaviors, maintaining\neffective human oversight of applications built with AutoGen agents will become importan

In [24]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

model = ChatOpenAI(model = 'gpt-4o-mini')
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("autogent이 뭐야?")

'AutoGen refers to a system that allows for autonomous interaction between AI agents and humans to solve problems collaboratively. It supports workflows where human oversight can either be integrated or optional, ensuring effective problem-solving. The system emphasizes the need for fail-safes in high-risk applications to avoid potential issues arising from high levels of autonomy.'